Recurrent Neural Network model for classification in sequences
====

In this example we will use the RNNModel to set up an experiment over the MINST dataset included in TensorFlow. Based in https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb

To classify images using a reccurent neural network, we consider every image row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 steps for every sample.

In [1]:
%matplotlib inline

In [2]:
# add parent directory to python path
import sys
sys.path.append('../')

In [3]:
import numpy
import os
import urllib
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import utils

In [4]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("downloads/MNIST_data/", one_hot=True)

Extracting downloads/MNIST_data/train-images-idx3-ubyte.gz
Extracting downloads/MNIST_data/train-labels-idx1-ubyte.gz
Extracting downloads/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting downloads/MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
import dataset
dataset = reload(dataset)

samples = 1

class MNSTDataset(dataset.SimpleDataset):
    def create_from_matrixes(self, mnist, unused=None, unused2=None):
        """Creates the dataset from a matrix and the set of indices.

        Args:
            matrix (:obj: iterable): the Mnist dataset.
        """
        self.datasets['train'] = mnist.train
        self.datasets['test'] = mnist.test
        self.datasets['validation'] = mnist.validation
        
        # All sequences have the same lenght
        self.sequences_lenghts = 28
    
    def next_batch(self, batch_size, partition_name='train', pad_sequences=False, max_sequence_length=None):
        """Generates batches of instances and labels from a partition.

        If the size of the partition is exceeded, the partition and the labels
        are shuffled to generate further batches.

        Args:
            batch_size (int): the size of each batch to generate.
            partition_name (str): the name of the partition to create the
                batches from.
        """
        instances, labels = self.datasets[partition_name].next_batch(batch_size)
        instances = instances.reshape((batch_size, self.feature_vector_size, self.sequences_lenghts))
        lenghts = numpy.zeros((batch_size,)) + self.sequences_lenghts
        return instances, labels, lenghts
    
    def classes_num(self, unused=None):
        """Returns the number of unique classes in a partition of the dataset.

        Returns:
            int: the number of uniqu classes.
        """
        return 10
    
    def traverse_dataset(self, batch_size, partition_name):
        """Generates batches of instances and labels from a partition.

        The iterator ends when the dataset has been entirely returned.

        Args:
            batch_size (int): the size of each batch to generate.
            partition_name (str): the name of the partition to create the
                batches from.

        Yields:
            A tuple (instances, lenghts, labels) of batch_size, except in the last
            iteration where the size can be smaller.
        """
        total_elements = 0
        while True:
            this_batch = batch_size
            if total_elements + batch_size >= self.datasets[partition_name].num_examples:
                this_batch = self.datasets[partition_name].num_examples - total_elements
            instances, labels = self.datasets[partition_name].next_batch(this_batch)
            instances = instances.reshape((this_batch, self.feature_vector_size, self.sequences_lenghts))
            lenghts = numpy.zeros((this_batch,)) + self.sequences_lenghts
            total_elements += this_batch
            yield instances, labels, lenghts
            if total_elements >= self.datasets[partition_name].num_examples:
                break
    
    def get_labels(self, partition_name='train'):
        """Returns the labels of the partition
        Args:
            partition_name (str): the name of the partition to get the
                labels from.

        Returns:
            An iterable with labels.
        """
        return numpy.argmax(self.datasets[partition_name].labels, axis=1)
    
    @property
    def labels_type(self):
        return tf.float64
    
    @property
    def feature_vector_size(self):
        return 28
    
    def num_examples(self, partition_name='train'):
        return self.datasets[partition_name].num_examples

In [15]:
mnist_dataset = MNSTDataset()
mnist_dataset.create_from_matrixes(mnist)

In [58]:
from models import lstm, mlp
mlp = reload(mlp)
lstm = reload(lstm)

class MnistLSTMModel(lstm.LSTMModel):
    """LSTM Model that handles labels as one hot encodings."""
    
    def _build_loss(self, logits):
        """Calculates the loss from the logits and the labels.

        Args:
            logits: Logits tensor, float - [self.batch_size, NUM_CLASSES].
            labels_placeholder: Labels tensor, float - [batch_size, NUM_CLASSES]

        Returns:

        """
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=self.labels_placeholder, name='cross_entropy')
        return tf.reduce_mean(cross_entropy, name='cross_entropy_mean_loss')
    
    def _build_inputs(self):
        """Generate placeholder variables to represent the input tensors."""
        # Placeholder for the inputs in a given iteration.
        self.instances_placeholder = tf.placeholder(
            tf.float32,
            (None, None, self.dataset.feature_vector_size),
            name='sequences_placeholder')
        
        self.lengths_placeholder = tf.placeholder(
            tf.int32, (None, ), name='lengths_placeholder')

        # Labels are now a one-hot encoding
        self.labels_placeholder = tf.placeholder(
            self.dataset.labels_type, (None, self.dataset.classes_num()),
            name='labels_placeholder')
    
    def _build_evaluation(self, logits):
        """Evaluate the quality of the logits at predicting the label.

        Args:
            logits: Logits tensor, float - [batch_size, NUM_CLASSES].
            labels: Labels tensor, int32 - [batch_size, NUM_CLASSES].

        Returns:
            A scalar int32 tensor with the number of examples (out of batch_size)
            that were predicted correctly.
        """
        correct = tf.equal(tf.argmax(logits,1), tf.argmax(self.labels_placeholder,1))
        return tf.reduce_sum(tf.cast(correct, tf.int32))
    
    def _build_train_operation(self, loss):
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        return optimizer.minimize(loss)
    
    def predict(self, partition_name):                                           
        predictions = []                                                         
        true = []                                                                
        with self.graph.as_default():                                            
            for instances, labels, lengths in self.dataset.traverse_dataset(     
                   self.batch_size, partition_name):                            
                feed_dict = {                                                    
                    self.instances_placeholder: instances,                       
                    self.labels_placeholder: labels,                             
                    self.lengths_placeholder: lengths                            
                }                                                                
                predictions.extend(self.sess.run(self.predictions,               
                                                 feed_dict=feed_dict))           
                true.append(numpy.argmax(labels, axis=1))                        
        return numpy.array(predictions), numpy.concatenate(true)

We can now create the dataset using the extracted instances and labels

In [59]:
# Remove previous directory
import shutil
logs_dirname = '../../results/examples/mnist/'
try:
    shutil.rmtree(logs_dirname)
except OSError:
    pass

utils.safe_mkdir(logs_dirname)

In [60]:
import experiment
experiment = reload(experiment)

config = {
    'model': MnistLSTMModel,
    'model_arguments': {'hidden_layer_size': 128, 'batch_size': 128,
                        'logs_dirname': None,
                        'log_values': 500, 'training_epochs': 500, 'max_num_steps': 28,
                        'learning_rate': 0.001}
}
mnist_experiment = experiment.Experiment(mnist_dataset, config=config)

In [61]:
tf.reset_default_graph()
mnist_experiment.run()

INFO:root:
	Precision	Recall	F1 Score	Support
Class 0	0.988775510204	0.903075489282	0.943984413054	1073.0
Class 1	0.984140969163	0.980684811238	0.982409850484	1139.0
Class 2	0.956395348837	0.935545023697	0.945855294681	1055.0
Class 3	0.946534653465	0.876260311641	0.910042836744	1091.0
Class 4	0.923625254582	0.936983471074	0.930256410256	968.0
Class 5	0.924887892377	0.94070695553	0.932730356133	877.0
Class 6	0.942588726514	0.966809421842	0.954545454545	934.0
Class 7	0.939688715953	0.947058823529	0.943359375	1020.0
Class 8	0.797741273101	0.941818181818	0.863813229572	825.0
Class 9	0.919722497522	0.911591355599	0.915638875185	1018.0


Let's print the accuracy

In [62]:
predictions, true = mnist_experiment.model.predict('test')

In [63]:
print predictions, numpy.unique(predictions, return_counts=True)

[1 8 1 ..., 2 3 4] (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([1073, 1139, 1055, 1091,  968,  877,  934, 1020,  825, 1018]))


In [64]:
from sklearn import metrics
metrics.accuracy_score(true, predictions)

0.9335